In [ ]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Cargar las variables de entorno
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

# Configurar el caché
requests_cache.install_cache('news_cache', expire_after=timedelta(hours=1))

# Lista de empresas
empresas = [
    "Altitude Investment Management",
    "Alinga Capital",
    "Aurora Cannabis",
    "acreageholdings",
    "4 Front Ventures",
    "Barenberg Capital Partners",
    "Bayline Capital Partners",
    "Big Rock",
    "Bullpen Capital",
    "cannaangelsllc.com",
    "cannahedge.com",
    "Canalis Capital",
    "Cannabis Capital",
    "Canopy Rivers",
    "canopyboulder",
    "Charlottes Web Holdings",
    "casaverdecapital",
    "canopygrowth",
    "Compass Cannabis Clinic",
    "Constellation Brands",
    "Craft Ventures",
    "Cusp Capital",
    "Capital 8 Group",
    "First Trust Portafolio",
    "FundRx",
    "KES7 Capital",
    "Krauter Group",
    "KushCo Holdings",
    "LaMarch Capital",
    "Lerer Hippeau Ventures",
    "Mainline Investment Partners",
    "Mazakali",
    "Medmen",
    "Mentor Capital",
    "Merida Capital Partners",
    "Miracle Mile Advisors",
    "Montegra Capital",
    "muse capital",
    "Navy Capital",
    "Nosara Capital Growthinc",
    "Phyto Partners",
    "Privateer Holdings",
    "Rose Capital",
    "Salveo Capital",
    "Green Acre Capital",
    "Green Growth Investments",
    "Green Leaf Investments",
    "HIGH TIMES",
    "Initiative Capital",
    "Supreme Cannabis",
    "The Arcview Group",
    "Tress Capital",
    "High Tide",
    "Crowe",
    "Flower Hire",
    "Primary Growth Partners",
    "CHICAGO ATLANTIC GROUP",
    "GREEN SPACE ACCOUNTING",
    "Merida Capital Holdings",
    "Fluence",
    "Woodcrick Ventures"
]

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 7 días)
from_date = (datetime.now() - timedelta(days=15)).strftime('%Y-%m-%d')

news_data = {}

for empresa in empresas:
    print(f"Buscando noticias para: {empresa}")
    params = {
        'q': empresa,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',  # Idioma de las noticias
        'from': from_date  # Fecha desde la cual buscar noticias
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        news_data[empresa] = []
        for article in articles:
            title = article['title']
            link = article['url']
            snippet = article['description']
            published_at = article['publishedAt']
            news_data[empresa].append(
                {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
        print(f"Encontradas {len(articles)} noticias para {empresa}")
    except requests.exceptions.RequestException as e:
        news_data[empresa] = f"Error fetching data: {str(e)}"
        print(f"Error al buscar noticias para {empresa}: {str(e)}")

# Guardar los resultados en un archivo JSON
with open('news_data.json', 'w') as f:
    json.dump(news_data, f, indent=4)

print("Noticias guardadas en news_data.json")


Procesar JSON y convertirlo a texto

In [ ]:
import json
import os

def process_json_file(input_filename):
    # Leer el archivo JSON
    with open(input_filename, 'r') as file:
        data = json.load(file)

    # Iterar sobre los datos JSON y extraer los elementos requeridos
    extracted_elements = []

    # Extraer títulos y descripciones de los artículos
    for company, articles in data.items():
        for article in articles:
            title = article['title']
            snippet = article['snippet']
            link = article['url']
            extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

    # Obtener el directorio del archivo de entrada
    output_dir = os.path.dirname(input_filename)

    # Construir el nombre del archivo de salida
    output_filename = os.path.join(output_dir, "processed_articles.txt")

    # Escribir los elementos extraídos en el archivo de salida
    with open(output_filename, 'w') as output_file:
        for element in extracted_elements:
            # Añadir dos saltos de línea para la separación
            output_file.write(element + "\n\n")

# Llamar a la función de procesamiento
process_json_file('news_data.json')


In [ ]:
# Usar Langchain para procesar el texto con LLM
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Cargar el archivo de texto procesado
text_file = "processed_articles.txt"
loader = TextLoader(text_file)
documents = loader.load()

# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [22]:
import pprint

# Lista de preguntas para identificar similitudes y repetición de temas entre noticias
questions = [
    "¿Cuáles son los titulos con similitudes entre las noticias recientes sobre la empresa?",
    "¿Qué titulos tienen noticias se repiten entre los diferentes artículos sobre la empresa?",
    "¿Qué titulos de noticias mencionan los mismos eventos o desarrollos sobre la empresa?",
    "¿Qué titulos de noticias discuten los mismos competidores en relación con la empresa?"
]

# Almacenar las respuestas en una lista
responses = []

for empresa in empresas:
    print(f"Analizando preguntas para: {empresa}")
    for j, query in enumerate(questions):
        full_query = f"{query} {empresa}"
        response = qa_chain({"query": full_query})
        responses.append({
            "empresa": empresa,
            "query": query,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {empresa}")

# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(responses)



Analizando preguntas para: Altitude Investment Management
Pregunta 1/4 analizada para Altitude Investment Management
Pregunta 2/4 analizada para Altitude Investment Management
Pregunta 3/4 analizada para Altitude Investment Management
Pregunta 4/4 analizada para Altitude Investment Management
Analizando preguntas para: Alinga Capital
Pregunta 1/4 analizada para Alinga Capital
Pregunta 2/4 analizada para Alinga Capital
Pregunta 3/4 analizada para Alinga Capital


KeyboardInterrupt: 

In [23]:

# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(responses)

[   {   'empresa': 'Altitude Investment Management',
        'query': '¿Cuáles son los titulos con similitudes entre las noticias '
                 'recientes sobre la empresa?',
        'response': 'Lo siento, no dispongo de información sobre noticias '
                    'recientes relacionadas con Altitude Investment '
                    'Management.'},
    {   'empresa': 'Altitude Investment Management',
        'query': '¿Qué titulos tienen noticias se repiten entre los diferentes '
                 'artículos sobre la empresa?',
        'response': 'No hay información disponible sobre noticias relacionadas '
                    'con Altitude Investment Management en los fragmentos '
                    'proporcionados. Lo siento, no puedo proporcionar los '
                    'títulos de noticias que se repiten para esa empresa '
                    'específica.'},
    {   'empresa': 'Altitude Investment Management',
        'query': '¿Qué titulos de noticias mencionan los mi

In [8]:
import requests
import json
from datetime import datetime, timedelta
import pprint

# Tu clave de API de NewsAPI
api_key = '1bae54a7ad13440cb6ea9e5bd059ce33'

# Tema de interés
tema = "cannabis"

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 7 días)
from_date = (datetime.now() - timedelta(days=15)).strftime('%Y-%m-%d')

print(f"Buscando noticias sobre: {tema}")
params = {
    'q': tema,
    'apiKey': api_key,
    'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
    'language': 'en',  # Idioma de las noticias
    'from': from_date  # Fecha desde la cual buscar noticias
}

response = requests.get(url, params=params)
if response.status_code == 200:
    articles = response.json().get('articles', [])
    news_data = []
    for article in articles:
        title = article['title']
        link = article['url']
        snippet = article['description']
        published_at = article['publishedAt']
        news_data.append(
            {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
    print(f"Encontradas {len(articles)} noticias sobre {tema}")
else:
    news_data = f"Error fetching data: {response.status_code}"
    print(
        f"Error al buscar noticias sobre {tema}: {response.status_code}")

# Guardar los resultados en un archivo JSON
with open('cannabis_news_data.json', 'w') as f:
    json.dump(news_data, f, indent=4)

print("Noticias guardadas en cannabis_news_data.json")


Buscando noticias sobre: cannabis
Error al buscar noticias sobre cannabis: 429
Noticias guardadas en cannabis_news_data.json


In [7]:
import json
import os

def process_json_file(input_filename):
    # Leer el archivo JSON
    with open(input_filename, 'r') as file:
        data = json.load(file)

    # Iterar sobre los datos JSON y extraer los elementos requeridos
    extracted_elements = []

    # Extraer títulos y descripciones de los artículos
    for article in data:
        title = article['title']
        snippet = article['snippet']
        link = article['link']
        extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

    # Obtener el directorio del archivo de entrada
    output_dir = os.path.dirname(input_filename)

    # Construir el nombre del archivo de salida
    output_filename = os.path.join(output_dir, "processed_cannabis_articles.txt")

    # Escribir los elementos extraídos en el archivo de salida
    with open(output_filename, 'w') as output_file:
        for element in extracted_elements:
            # Añadir dos saltos de línea para la separación
            output_file.write(element + "\n\n")

# Llamar a la función de procesamiento
process_json_file('cannabis_news_data.json')


TypeError: string indices must be integers, not 'str'

In [6]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Cargar el archivo de texto procesado
text_file = "processed_cannabis_articles.txt"
loader = TextLoader(text_file)
documents = loader.load()

# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.333, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())



RuntimeError: Error loading processed_cannabis_articles.txt

In [5]:
# Lista de preguntas para identificar similitudes y repetición de temas entre noticias
questions = questions = [
    "¿Cuáles son los títulos de las noticias más populares del cannabis ?",
    "¿Cuáles son los títulos de las noticias mas de moda en el cannabis?",
    "¿Cuáles son los títulos de las noticias sobre cannabis discutidas en múltiples medios?",
    "¿Cuales son los titulos de noticias sobre el cannabis más populares relacionados a las finanzas?",
    "¿Cuales son los titulos de noticias sobre el cannabis  más populares relacionados a la salud?",
    "¿Cuales son los titulos de noticias sobre el cannabis  más populares relacionados a la legalización?",
    "¿Cuales son los titulos de noticias sobre el cannabis  más populares relacionados a la tecnología?",

]

# Almacenar las respuestas en una lista
responses = []

for query in questions:
    full_query = f"{query}"
    response = qa_chain({"query": full_query})
    responses.append({
        "tema": tema,
        "query": query,
        "response": response["result"]
    })

# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=8)
pp.pprint(responses)

NameError: name 'qa_chain' is not defined

Código para obtener noticias y manejar caché

In [3]:
!pip install requests-cache


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 570.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.9 MB/s eta 0:00:00


In [13]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Cargar las variables de entorno

api_key = "5374a9b653a047bc9b0a42f809a8445b"

# Configurar el caché con una expiración de 1 día
requests_cache.install_cache('news_cache', expire_after=timedelta(days=1))

# Lista de empresas
empresas = [
    "Altitude Investment Management",
    "Alinga Capital",
    "Aurora Cannabis",
    "acreageholdings",
    "4 Front Ventures",
    "Barenberg Capital Partners",
    "Bayline Capital Partners",
    "Big Rock",
    "Bullpen Capital",
    "cannaangelsllc.com",
    "cannahedge.com",
    "Canalis Capital",
    "Cannabis Capital",
    "Canopy Rivers",
    "canopyboulder",
    "Charlottes Web Holdings",
    "casaverdecapital",
    "canopygrowth",
    "Compass Cannabis Clinic",
    "Constellation Brands",
    "Craft Ventures",
    "Cusp Capital",
    "Capital 8 Group",
    "First Trust Portafolio",
    "FundRx",
    "KES7 Capital",
    "Krauter Group",
    "KushCo Holdings",
    "LaMarch Capital",
    "Lerer Hippeau Ventures",
    "Mainline Investment Partners",
    "Mazakali",
    "Medmen",
    "Mentor Capital",
    "Merida Capital Partners",
    "Miracle Mile Advisors",
    "Montegra Capital",
    "muse capital",
    "Navy Capital",
    "Nosara Capital Growthinc",
    "Phyto Partners",
    "Privateer Holdings",
    "Rose Capital",
    "Salveo Capital",
    "Green Acre Capital",
    "Green Growth Investments",
    "Green Leaf Investments",
    "HIGH TIMES",
    "Initiative Capital",
    "Supreme Cannabis",
    "The Arcview Group",
    "Tress Capital",
    "High Tide",
    "Crowe",
    "Flower Hire",
    "Primary Growth Partners",
    "CHICAGO ATLANTIC GROUP",
    "GREEN SPACE ACCOUNTING",
    "Merida Capital Holdings",
    "Fluence",
    "Woodcrick Ventures"
]

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 7 días)
from_date = (datetime.now() - timedelta(days=10)).strftime('%Y-%m-%d')

news_data = {}

for empresa in empresas:
    print(f"Buscando noticias para: {empresa}")
    params = {
        'q': empresa,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',  # Idioma de las noticias
        'from': from_date  # Fecha desde la cual buscar noticias
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        news_data[empresa] = []
        for article in articles:
            title = article['title']
            link = article['url']
            snippet = article['description']
            published_at = article['publishedAt']
            news_data[empresa].append(
                {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
        print(f"Encontradas {len(articles)} noticias para {empresa}")
    except requests.exceptions.RequestException as e:
        news_data[empresa] = f"Error fetching data: {str(e)}"
        print(f"Error al buscar noticias para {empresa}: {str(e)}")
# Filtrar empresas sin noticias
news_data = {empresa: articles for empresa, articles in news_data.items() if articles}
# No guardar en archivo JSON, continuar con el procesamiento en memoria


Buscando noticias para: Altitude Investment Management
Encontradas 18 noticias para Altitude Investment Management
Buscando noticias para: Alinga Capital
Encontradas 0 noticias para Alinga Capital
Buscando noticias para: Aurora Cannabis
Encontradas 6 noticias para Aurora Cannabis
Buscando noticias para: acreageholdings
Encontradas 0 noticias para acreageholdings
Buscando noticias para: 4 Front Ventures
Encontradas 20 noticias para 4 Front Ventures
Buscando noticias para: Barenberg Capital Partners
Encontradas 0 noticias para Barenberg Capital Partners
Buscando noticias para: Bayline Capital Partners
Encontradas 0 noticias para Bayline Capital Partners
Buscando noticias para: Big Rock
Encontradas 100 noticias para Big Rock
Buscando noticias para: Bullpen Capital
Encontradas 4 noticias para Bullpen Capital
Buscando noticias para: cannaangelsllc.com
Encontradas 0 noticias para cannaangelsllc.com
Buscando noticias para: cannahedge.com
Encontradas 0 noticias para cannahedge.com
Buscando not

In [14]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pprint

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Procesar los datos en memoria
extracted_elements = []

for company, articles in news_data.items():
    if isinstance(articles, list):
        for article in articles:
            title = article.get('title', 'No title')
            snippet = article.get('snippet', 'No snippet')
            link = article.get('link', 'No link')
            extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

# Filtrar empresas sin noticias
filtered_elements = [element for element in extracted_elements if 'Title: No title' not in element]

# Crear documentos para Langchain
documents = [Document(page_content=element) for element in filtered_elements]


# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-4", temperature=0.4, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [16]:

# Lista de preguntas para identificar similitudes y repetición de temas entre noticias
questions = [
    "¿De que se habla en las noticias recientes sobre la empresa?",
    "¿Cuáles son los titulos con similitudes entre las noticias recientes sobre la empresa?",
    "¿Qué titulos tienen noticias se repiten entre los diferentes artículos sobre la empresa?",
    "¿Qué titulos de noticias mencionan los mismos eventos o desarrollos sobre la empresa?",
    "¿Qué titulos de noticias discuten los mismos competidores en relación con la empresa?"
]

# Almacenar las respuestas en una lista
responses = []

for empresa in news_data:
    print(f"Analizando preguntas para: {empresa}")
    for j, query in enumerate(questions):
        full_query = f"{query} {empresa}"
        response = qa_chain({"query": full_query})
        responses.append({
            "empresa": empresa,
            "query": query,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {empresa}")



Analizando preguntas para: Altitude Investment Management
Pregunta 1/5 analizada para Altitude Investment Management
Pregunta 2/5 analizada para Altitude Investment Management
Pregunta 3/5 analizada para Altitude Investment Management
Pregunta 4/5 analizada para Altitude Investment Management
Pregunta 5/5 analizada para Altitude Investment Management
Analizando preguntas para: Aurora Cannabis
Pregunta 1/5 analizada para Aurora Cannabis
Pregunta 2/5 analizada para Aurora Cannabis
Pregunta 3/5 analizada para Aurora Cannabis
Pregunta 4/5 analizada para Aurora Cannabis
Pregunta 5/5 analizada para Aurora Cannabis
Analizando preguntas para: 4 Front Ventures
Pregunta 1/5 analizada para 4 Front Ventures
Pregunta 2/5 analizada para 4 Front Ventures
Pregunta 3/5 analizada para 4 Front Ventures
Pregunta 4/5 analizada para 4 Front Ventures
Pregunta 5/5 analizada para 4 Front Ventures
Analizando preguntas para: Big Rock
Pregunta 1/5 analizada para Big Rock
Pregunta 2/5 analizada para Big Rock
Pregu

In [12]:
# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=10)
pp.pprint(responses)

[         {         'empresa': 'Altitude Investment Management',
                    'query': '¿De que se habla en las noticias recientes sobre '
                             'la empresa?',
                    'response': 'Lo siento, no tengo información específica '
                                'sobre noticias recientes relacionadas con '
                                'Altitude Investment Management. Te '
                                'recomendaría buscar en fuentes de noticias '
                                'financieras o en el sitio web de la empresa '
                                'para obtener la información más actualizada.'},
          {         'empresa': 'Altitude Investment Management',
                    'query': '¿Cuáles son los titulos con similitudes entre '
                             'las noticias recientes sobre la empresa?',
                    'response': 'Lo siento, pero no tengo información sobre '
                                'noticias recientes r

In [18]:
import requests
import requests_cache
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Cargar las variables de entorno

api_key = '5374a9b653a047bc9b0a42f809a8445b'

# Configurar el caché con una expiración de 1 día
requests_cache.install_cache('news_cache', expire_after=timedelta(days=1))

# Lista de temas relacionados con cannabis
temas = [
    "Cannabis legalization",
    "Medical marijuana",
    "Cannabis industry",
    "Cannabis investment",
    "Cannabis stocks",
    "Cannabis research",
    "Cannabis news",
    "Cannabis policy",
    "Cannabis business",
    "Cannabis culture",
    "Psychedelics research",
    "Psychedelics industry",
    "Psychedelics investment",
    "Psychedelics news",
    "Psychedelics policy",
    "Psychedelics business",
    "Psychedelics culture"
]

# Endpoint de la API
url = 'https://newsapi.org/v2/everything'

# Definir la fecha de inicio (últimos 7 días)
from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

news_data = {}

for tema in temas:
    print(f"Buscando noticias para: {tema}")
    params = {
        'q': tema,
        'apiKey': api_key,
        'sortBy': 'publishedAt',  # Ordenar por fecha de publicación
        'language': 'en',  # Idioma de las noticias
        'from': from_date  # Fecha desde la cual buscar noticias
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        if articles:
            news_data[tema] = []
            for article in articles:
                title = article['title']
                link = article['url']
                snippet = article['description']
                published_at = article['publishedAt']
                news_data[tema].append(
                    {'title': title, 'link': link, 'snippet': snippet, 'published_at': published_at})
            print(f"Encontradas {len(articles)} noticias para {tema}")
        else:
            print(f"No se encontraron noticias para {tema}")
    except requests.exceptions.RequestException as e:
        print(f"Error al buscar noticias para {tema}: {str(e)}")

# Filtrar temas sin noticias
news_data = {tema: articles for tema, articles in news_data.items() if articles}

# No guardar en archivo JSON, continuar con el procesamiento en memoria


Buscando noticias para: Cannabis legalization
Encontradas 20 noticias para Cannabis legalization
Buscando noticias para: Medical marijuana
Encontradas 57 noticias para Medical marijuana
Buscando noticias para: Cannabis industry
Encontradas 96 noticias para Cannabis industry
Buscando noticias para: Cannabis investment
Encontradas 33 noticias para Cannabis investment
Buscando noticias para: Cannabis stocks
Encontradas 30 noticias para Cannabis stocks
Buscando noticias para: Cannabis research
Encontradas 64 noticias para Cannabis research
Buscando noticias para: Cannabis news
Encontradas 100 noticias para Cannabis news
Buscando noticias para: Cannabis policy
Encontradas 52 noticias para Cannabis policy
Buscando noticias para: Cannabis business
Encontradas 100 noticias para Cannabis business
Buscando noticias para: Cannabis culture
Encontradas 18 noticias para Cannabis culture


In [20]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pprint

# Cargar las variables de entorno
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')

# Procesar los datos en memoria
extracted_elements = []

for tema, articles in news_data.items():
    for article in articles:
        title = article.get('title', 'No title')
        snippet = article.get('snippet', 'No snippet')
        link = article.get('link', 'No link')
        extracted_elements.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")

# Crear documentos para Langchain
documents = [Document(page_content=element) for element in extracted_elements]

# Dividir los documentos en trozos más pequeños
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Crear embeddings y base de datos Chroma
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
db = Chroma.from_documents(docs, embeddings)

# Inicializar el modelo y el retriever
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_key)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [21]:
questions = [
    "¿Cuáles son los titulos con similitudes entre las noticias recientes sobre el tema?",
    "¿Qué titulos de noticias se repiten entre los diferentes artículos sobre el tema?",
    "¿Qué titulos de noticias mencionan los mismos eventos o desarrollos sobre el tema?",
    "¿Qué titulos de noticias discuten los mismos actores o competidores en relación con el tema?"
]

# Almacenar las respuestas en una lista
responses = []

for tema in news_data:
    print(f"Analizando preguntas para: {tema}")
    for j, query in enumerate(questions):
        full_query = f"{query} {tema}"
        response = qa_chain({"query": full_query})
        responses.append({
            "tema": tema,
            "query": query,
            "response": response["result"]
        })
        print(f"Pregunta {j+1}/{len(questions)} analizada para {tema}")



Analizando preguntas para: Cannabis legalization
Pregunta 1/4 analizada para Cannabis legalization
Pregunta 2/4 analizada para Cannabis legalization
Pregunta 3/4 analizada para Cannabis legalization
Pregunta 4/4 analizada para Cannabis legalization
Analizando preguntas para: Medical marijuana
Pregunta 1/4 analizada para Medical marijuana
Pregunta 2/4 analizada para Medical marijuana
Pregunta 3/4 analizada para Medical marijuana
Pregunta 4/4 analizada para Medical marijuana
Analizando preguntas para: Cannabis industry
Pregunta 1/4 analizada para Cannabis industry
Pregunta 2/4 analizada para Cannabis industry
Pregunta 3/4 analizada para Cannabis industry
Pregunta 4/4 analizada para Cannabis industry
Analizando preguntas para: Cannabis investment
Pregunta 1/4 analizada para Cannabis investment
Pregunta 2/4 analizada para Cannabis investment
Pregunta 3/4 analizada para Cannabis investment
Pregunta 4/4 analizada para Cannabis investment
Analizando preguntas para: Cannabis stocks
Pregunta 1/

In [23]:
# Imprimir las respuestas obtenidas con pprint
pp = pprint.PrettyPrinter(indent=8)
pp.pprint(responses)

[       {       'query': '¿Cuáles son los titulos con similitudes entre las '
                         'noticias recientes sobre el tema?',
                'response': 'Los títulos con similitudes entre las noticias '
                            'recientes sobre el tema de la legalización del '
                            'cannabis son:\n'
                            '\n'
                            '1. "Thai Legalization Advocates Rally as '
                            'Government Moves Toward Cannabis Ban"',
                'tema': 'Cannabis legalization'},
        {       'query': '¿Qué titulos de noticias se repiten entre los '
                         'diferentes artículos sobre el tema?',
                'response': 'Los títulos de noticias que se repiten entre los '
                            'diferentes artículos sobre el tema de la '
                            'legalización del cannabis son:\n'
                            '\n'
                            '- "Thai Legalizatio